In [1]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [2]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [3]:
Loading_Data=pd.read_csv('Loading Data for Usage - Gross Loads.csv')


In [4]:
Loading_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97576 entries, 0 to 97575
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   BUS_BILLING_ACCOUNT_NUM  97576 non-null  int64 
 1   SHS_RGU_RPT_DT           97576 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [5]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.Loading_data'

bq_table_instance= bq_client.load_table_from_dataframe(Loading_Data, Table_BQ,job_config=config)

In [10]:
Query='''



with base_data as (
select BUS_BILLING_ACCOUNT_NUM as BAN,SHS_RGU_RPT_DT  from `divgpras-pr-579355.SHS.Loading_data`
)


, ADC_data_Sep_2022 as 

(

SELECT distinct
    BAN as BAN,
    avg(Login_consistency) as Login_consistency_sep ,
    avg(number_of_login_days) as Login_days_sep,
    avg(Arming_Consistency) as Arming_consistency_sep,
    1 as Sept_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-09-01'
  group by BAN
    
)

, ADC_data_oct_2022 as 

(

SELECT distinct
    BAN as BAN,
    avg(Login_consistency) as Login_consistency_Oct ,
    avg(number_of_login_days) as Login_days_Oct,
    avg(Arming_Consistency) as Arming_consistency_Oct,
    1 as Oct_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-10-01'
  group by BAN
    
)


, ADC_data_Nov_2022 as 

(

SELECT distinct
    BAN as BAN,
    avg(Login_consistency) as Login_consistency_Nov ,
    avg(number_of_login_days) as Login_days_Nov,
    avg(Arming_Consistency) as Arming_consistency_Nov,
    1 as Nov_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-11-01'
  group by BAN
    
)



, ADC_data_Dec_2022 as 

(

SELECT distinct
    BAN as BAN,
    avg(Login_consistency) as Login_consistency_Dec ,
    avg(number_of_login_days) as Login_days_Dec,
    avg(Arming_Consistency) as Arming_consistency_Dec,
    1 as Dec_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-12-01'
  group by BAN
    
)


, ADC_data_Jan_2023 as 

(

SELECT distinct
    BAN as BAN,
    avg(Login_consistency) as Login_consistency_Jan ,
    avg(number_of_login_days) as Login_days_Jan,
    avg(Arming_Consistency) as Arming_consistency_Jan,
    1 as Jan_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2023-01-01'
  group by BAN
    
)





select a.*
,b.Login_consistency_sep,b.Login_days_sep,b.Arming_consistency_sep,b.Sept_data_flag
,c.Login_consistency_Oct,c.Login_days_Oct,c.Arming_consistency_Oct,c.Oct_data_flag
,d.Login_consistency_Nov,d.Login_days_Nov,d.Arming_consistency_Nov,d.Nov_data_flag
,e.Login_consistency_Dec,e.Login_days_Dec,e.Arming_consistency_Dec,e.Dec_data_flag
,f.Login_consistency_Jan,f.Login_days_Jan,f.Arming_consistency_Jan,f.Jan_data_flag
from base_data a
left join ADC_data_Sep_2022 b
on a.BAN=b.BAN
left join ADC_data_oct_2022 c
on a.BAN=c.BAN
left join ADC_data_Nov_2022 d
on a.BAN=d.BAN
left join ADC_data_Dec_2022 e
on a.BAN=e.BAN
left join ADC_data_Jan_2023 f
on a.BAN=f.BAN
order by BAN


'''

In [11]:
DF=extract_bq_data(bq_client, sql=Query)

In [9]:
# DF_2022=extract_bq_data(bq_client, sql=Query)

In [12]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97576 entries, 0 to 97575
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BAN                     97576 non-null  Int64  
 1   SHS_RGU_RPT_DT          97576 non-null  object 
 2   Login_consistency_sep   16796 non-null  float64
 3   Login_days_sep          16796 non-null  float64
 4   Arming_consistency_sep  16796 non-null  float64
 5   Sept_data_flag          16796 non-null  Int64  
 6   Login_consistency_Oct   29214 non-null  float64
 7   Login_days_Oct          29214 non-null  float64
 8   Arming_consistency_Oct  29214 non-null  float64
 9   Oct_data_flag           29214 non-null  Int64  
 10  Login_consistency_Nov   41155 non-null  float64
 11  Login_days_Nov          41155 non-null  float64
 12  Arming_consistency_Nov  41155 non-null  float64
 13  Nov_data_flag           41155 non-null  Int64  
 14  Login_consistency_Dec   52803 non-null

In [13]:
DF.head(10)

,BAN,SHS_RGU_RPT_DT,Login_consistency_sep,Login_days_sep,Arming_consistency_sep,Sept_data_flag,Login_consistency_Oct,Login_days_Oct,Arming_consistency_Oct,Oct_data_flag,...,Arming_consistency_Nov,Nov_data_flag,Login_consistency_Dec,Login_days_Dec,Arming_consistency_Dec,Dec_data_flag,Login_consistency_Jan,Login_days_Jan,Arming_consistency_Jan,Jan_data_flag
0,123459524,2023-01-25 0:00:00,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,...,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>
1,123459649,2023-03-28 0:00:00,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,...,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>
2,123462230,2022-09-16 0:00:00,50.0,15.0,0.0,1,54.838710,17.0,0.000000,1,...,0.0,1,100.000000,30.0,0.000000,1,100.000000,30.0,0.000000,1
3,123462625,2023-01-31 0:00:00,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,...,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>
4,123464801,2023-02-25 0:00:00,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,...,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>
5,123466938,2022-12-05 0:00:00,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,...,NaN,<NA>,56.666667,17.0,3.225806,1,56.666667,17.0,3.225806,1
6,123467956,2022-10-06 0:00:00,NaN,NaN,NaN,<NA>,29.032258,9.0,13.333333,1,...,10.0,1,40.000000,12.0,6.451613,1,40.000000,12.0,6.451613,1
7,123469691,2022-10-19 0:00:00,NaN,NaN,NaN,<NA>,3.225806,1.0,43.333333,1,...,100.0,1,100.000000,30.0,100.000000,1,100.000000,30.0,100.000000,1
8,123471775,2022-11-25 0:00:00,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,...,0.0,1,33.333333,10.0,0.000000,1,33.333333,10.0,0.000000,1
9,123473465,2023-02-08 0:00:00,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,...,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>


In [14]:
DF.to_csv('Loading Data for Usage - Gross Loads_results.csv',index=False)